In [54]:
import face_recognition
import matplotlib.pyplot as plt
import json
from pathlib import Path
import os
import pandas as pd

In [55]:
json_path = '../data/soccernet/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/Labels-v2.json'
with open(json_path) as f:
    data = json.load(f)

annotations = data['annotations']
for i in range (0,len(annotations)):
    if(annotations[i]['gameTime'][0] == '2'):
        index=i
        break
an1 = annotations[:index]
an2 = annotations[index:]
actions = [an1,an2]

def frame_to_min(frame):
    s = frame // 2
    mins = s // 60
    sec = s % 60
    return str(mins)+':'+str(sec)

def get_frame(position):
    return (position//1000)*2


In [56]:
annotations[96]

{'gameTime': '1 - 44:49',
 'label': 'Clearance',
 'position': '2689819',
 'team': 'away',
 'visibility': 'visible'}

In [57]:
faces = pd.DataFrame(columns=['face_locations','current_frame','action_frame','action_position','action_name','action_time','half'])
match_path = Path('../data/soccernet/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley')
# falta: definir numero de frames atrás y delante desde fuera del loop

for half in range (0,1):
    frames_dir = match_path.joinpath(f'{half+1}_HQ/frames')
    action_list = actions[half]

    # going action by action
    for action in action_list[1:]:
        print(action)
        position = int(action['position'])
        frame_num = get_frame(position)
        
        # define first and last action frame and ensure that this frames exists
        if (frame_num-5>0):
            first_action_frame = frame_num-5
        else:
            first_action_frame = 1
        if (frame_num + 20 > len(os.listdir(frames_dir))):
            last_action_frame = len(os.listdir(frames_dir))
        else:
            last_action_frame = frame_num + 20
        
        # going frame by frame inside the action
        for i in range (first_action_frame, last_action_frame):
            frame_path = frames_dir.joinpath(f'{i:05}' +'.jpg')
            image = face_recognition.load_image_file(frame_path)
            face_locations = face_recognition.face_locations(image)

            if(len(face_locations)>0):
                faces = faces.append({'face_locations':face_locations,'current_frame':i, 'action_frame':frame_num,
                                      'action_position':position, 'action_name':action['label'], 'action_time':action['gameTime'],
                                      'half':half+1}, ignore_index=True)

            



{'gameTime': '1 - 02:13', 'label': 'Ball out of play', 'position': '133295', 'team': 'not applicable', 'visibility': 'visible'}
{'gameTime': '1 - 02:29', 'label': 'Throw-in', 'position': '149168', 'team': 'away', 'visibility': 'visible'}
{'gameTime': '1 - 02:36', 'label': 'Ball out of play', 'position': '156098', 'team': 'not applicable', 'visibility': 'visible'}
{'gameTime': '1 - 03:02', 'label': 'Corner', 'position': '182775', 'team': 'away', 'visibility': 'visible'}
{'gameTime': '1 - 04:06', 'label': 'Ball out of play', 'position': '246619', 'team': 'not applicable', 'visibility': 'visible'}
{'gameTime': '1 - 04:18', 'label': 'Throw-in', 'position': '258040', 'team': 'home', 'visibility': 'not shown'}
{'gameTime': '1 - 04:30', 'label': 'Shots on target', 'position': '270441', 'team': 'home', 'visibility': 'visible'}
{'gameTime': '1 - 04:31', 'label': 'Ball out of play', 'position': '271751', 'team': 'not applicable', 'visibility': 'visible'}
{'gameTime': '1 - 04:52', 'label': 'Corne

{'gameTime': '1 - 33:11', 'label': 'Ball out of play', 'position': '1991776', 'team': 'not applicable', 'visibility': 'visible'}
{'gameTime': '1 - 33:41', 'label': 'Corner', 'position': '2021651', 'team': 'away', 'visibility': 'visible'}
{'gameTime': '1 - 33:50', 'label': 'Ball out of play', 'position': '2030824', 'team': 'not applicable', 'visibility': 'visible'}
{'gameTime': '1 - 33:58', 'label': 'Throw-in', 'position': '2038287', 'team': 'away', 'visibility': 'visible'}
{'gameTime': '1 - 34:14', 'label': 'Ball out of play', 'position': '2054894', 'team': 'not applicable', 'visibility': 'visible'}
{'gameTime': '1 - 34:36', 'label': 'Throw-in', 'position': '2076398', 'team': 'away', 'visibility': 'visible'}
{'gameTime': '1 - 34:38', 'label': 'Ball out of play', 'position': '2078978', 'team': 'not applicable', 'visibility': 'visible'}
{'gameTime': '1 - 35:01', 'label': 'Clearance', 'position': '2101488', 'team': 'home', 'visibility': 'visible'}
{'gameTime': '1 - 35:57', 'label': 'Ball 

In [58]:
faces

,face_locations,current_frame,action_frame,action_position,action_name,action_time,half
0,"[(341, 1083, 563, 860)]",280,266,133295,Ball out of play,1 - 02:13,1
1,"[(217, 1083, 440, 860), (605, 1154, 713, 1047)]",317,298,149168,Throw-in,1 - 02:29,1
2,"[(217, 1083, 440, 860), (605, 1154, 713, 1047)]",317,312,156098,Ball out of play,1 - 02:36,1
3,"[(291, 984, 514, 761)]",318,312,156098,Ball out of play,1 - 02:36,1
4,"[(291, 959, 514, 736)]",319,312,156098,Ball out of play,1 - 02:36,1
...,...,...,...,...,...,...,...
346,"[(452, 291, 504, 239)]",5324,5308,2654007,Ball out of play,1 - 44:14,1
347,"[(188, 1498, 262, 1423)]",5326,5308,2654007,Ball out of play,1 - 44:14,1
348,"[(241, 1061, 562, 740)]",5373,5378,2689819,Clearance,1 - 44:49,1
349,"[(241, 1133, 562, 812)]",5374,5378,2689819,Clearance,1 - 44:49,1


In [59]:
faces.to_json('../Extracted Data/face_locations_1half.json',orient='records')